In [238]:
import numpy as np

In [239]:
print("For the sake of verification, data was generated from a Gaussian Mixture")
print("Give degree of data for generation (in positive integers)")
deg = int(input())
print("Number of modes for generation")
Nm = int(input())
print("Number of data points")
Nspm = int(input())

For the sake of verification, data was generated from a Gaussian Mixture
Give degree of data for generation (in positive integers)
2
Number of modes for generation
3
Number of data points
100


In [240]:
Ns = Nspm

Mean = []
Cov = []
data = []
Mix = np.linspace(10,100,Nm)
Mix = Mix/np.sum(Mix)
Mix = Mix.tolist()
for i in range(Nm):
    Mean.append(np.random.randint(1,10,deg))
    temp = np.random.randint(2,9,(deg,deg))
    Cov.append(np.matmul(temp,temp.T))
for i in range(Nspm):
    temp1 = 0
    for j in range(Nm):
        temp1 += Mix[j] * np.random.multivariate_normal(Mean[j],Cov[j],1)
    data.append(temp1)
data = np.array(data)
data = data.reshape(Ns,deg)
print(Mix)

[0.06060606060606061, 0.3333333333333333, 0.6060606060606061]


In [241]:
def gaus_mul_pdf(x,mean,cov,degree):
    x = x.reshape((degree,1))
    mean = mean.reshape((degree,1))
    ph = np.matmul(np.matmul(((x-mean).T),np.linalg.inv(cov)),(x-mean))
    temp = np.exp((-0.5)*ph)/np.sqrt((2*np.pi*np.linalg.det(cov)))
    temp = np.asscalar(temp)
#     print(temp.shape)
    return temp

In [242]:
def gam(mix,data_matrix,mean_matrix,cov_matrix,Nm,i,k,degree):
    temp = 0
    for w in range(Nm):
        temp = temp + mix[w]*gaus_mul_pdf(data_matrix[i],mean_matrix[w],cov_matrix[w],degree)
    temp1 = mix[k]*gaus_mul_pdf(data_matrix[i],mean_matrix[k],cov_matrix[k],degree)/temp
    return temp1

In [243]:
def Nk(mix,data_matrix,mean_matrix,cov_matrix,Nm,k,Ns,degree):
    temp = 0
    for w in range(Ns):
        temp = temp + gam(mix,data_matrix,mean_matrix,cov_matrix,Nm,w,k,degree)
    return temp

In [244]:
def log_likelihood(data,mean_matrix,cov_matrix,mix,deg,Ns):
    temp = 0
    temp1 = 0
    for i in range(Ns):
        for j in range(deg):
            temp += mix[j]*gaus_mul_pdf(data[i],mean_matrix[j],cov_matrix[j],deg)
        temp1 += np.log(np.array(temp))
#     print(temp1)
    return temp1

In [245]:
### Initial Parameter setting
# mix = np.full((1,Nm),1/Nm)
mix = []
cov_matrix = []
mean_matrix = []
for i in range(Nm):
    mean_matrix.append(np.random.randint(1,10,deg))
    temp = (np.random.randint(1,10,(deg,deg)))
    cov_matrix.append(np.matmul(temp,temp.T))
    mix.append(float(1.0/Nm))

In [246]:
# cov_matrix[0]
log_likelihood(data,mean_matrix,cov_matrix,mix,deg,Ns)

-197.56241932206342

In [247]:
print("Give log error delta")
error = float(input())
itera = 0
err_delta = 100.0
while(itera<100):
    new_mean_matrix = []
    new_cov_matrix = []
    new_mix = []
    log_o = log_likelihood(data,mean_matrix,cov_matrix,mix,deg,Ns)
#     print(mix)
    for k in range(Nm):
        meh = Nk(mix,data,mean_matrix,cov_matrix,Nm,k,Ns,deg)
        temp1 = 0
        temp2 = 0
        temp3 = 0
        for i in range(Ns):
            temp1 = np.outer((data[i]-mean_matrix[k]),(data[i]-mean_matrix[k]))
            temp2 += gam(mix,data,mean_matrix,cov_matrix,Nm,i,k,deg)*temp1
            temp3 += gam(mix,data,mean_matrix,cov_matrix,Nm,i,k,deg)*data[i]
        new_cov_matrix.append(temp2/meh)
        new_mean_matrix.append(temp3/meh)
        new_mix.append(meh/Ns)
    mean_matrix = new_mean_matrix
    cov_matrix = new_cov_matrix
    mix = new_mix
    log_n = log_likelihood(data,mean_matrix,cov_matrix,mix,deg,Ns)
    err_delta = np.abs(log_o - log_n)
#     print("### ",err_delta)
    print(log_o)
    itera += 1
    if err_delta<error:
        break

Give log error delta
1e-6
-197.56241932206342
-109.75443777620303
-101.15617929526474
-101.73842785158621
-102.33094281173301
-102.75881565595036
-103.06482806820436
-103.2884364633622
-103.4576068507747
-103.59045654349639
-103.69804067369671
-103.78660243987039
-103.85915689278555
-103.9165763279632
-103.95834055866445
-103.9830632485226
-103.98886637070997
-103.97365430021802
-103.93532981521135
-103.87198917240042
-103.78212828987617
-103.66488502264056
-103.52033192041831
-103.34981687795512
-103.15632186331294
-102.94477070393567
-102.7221735030671
-102.49747422094973
-102.28101240649086
-102.08364285106714
-101.91571677973621
-101.78617436502948
-101.701871266305
-101.66711415273504
-101.68338506519977
-101.7493177970339
-101.860964088508
-102.01224549501677
-102.19536472883185
-102.40092269267988
-102.61753144221507
-102.83085027680272
-103.02252996021394
-103.1716147532341
-103.26553761510212
-103.3235757907342
-103.39144118612765
-103.4869771755467
-103.59020284710942
-103.66

In [248]:
print(mix)
print(mean_matrix)
print(cov_matrix)

[0.6118270201556669, 0.0369040641786927, 0.3512689156656401]
[array([6.71043249, 9.52124662]), array([ 2.49696351, 12.53103327]), array([-0.38139597,  2.32486415])]
[array([[20.65239374, 15.34909121],
       [15.34909121, 16.80797957]]), array([[70.88354513, 63.10056258],
       [63.10056258, 56.25795188]]), array([[ 9.08349893,  8.03279183],
       [ 8.03279183, 13.7800045 ]])]
